In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import cv2

In [2]:
label_data = pd.read_csv('kaggle/train_study_level.csv')

In [3]:
label_data.shape

(6054, 5)

In [4]:
label2 = pd.read_csv('kaggle/train_image_level.csv')

In [5]:
label2.shape

(6334, 4)

In [6]:
label2.head(2)

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed


In [7]:
label2['StudyInstanceUID'] = label2['StudyInstanceUID'].apply(lambda x : x+'_study')

In [8]:
label_data = label_data.merge(label2,left_on='id',right_on='StudyInstanceUID',how='left')

In [9]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
0,00086460a852_study,0,1,0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study
1,000c9c05fd14_study,0,0,0,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study


In [10]:
label_data['id_x'].value_counts(ascending=False)

0fd2db233deb_study    9
a7335b2f9815_study    7
8943d1d85097_study    6
a0254bf8a96e_study    5
970c96c9fa5d_study    5
                     ..
1665788ce755_study    1
e13f252bf83c_study    1
2a8a246f1ebb_study    1
2e8e6d58eccd_study    1
3865c0d08f85_study    1
Name: id_x, Length: 6054, dtype: int64

In [11]:
label_data[label_data['id_x']=='0fd2db233deb_study']

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
375,0fd2db233deb_study,0,0,1,0,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
376,0fd2db233deb_study,0,0,1,0,05c063f5cef5_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
377,0fd2db233deb_study,0,0,1,0,156cb1f5c689_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
378,0fd2db233deb_study,0,0,1,0,26f643772090_image,"[{'x': 561.30269, 'y': 163.28797, 'width': 883...",opacity 1 561.30269 163.28797 1445.02377 1495....,0fd2db233deb_study
379,0fd2db233deb_study,0,0,1,0,4c414b793562_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
380,0fd2db233deb_study,0,0,1,0,a5a364383f34_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
381,0fd2db233deb_study,0,0,1,0,b121806162c3_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
382,0fd2db233deb_study,0,0,1,0,bee62c601ae9_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
383,0fd2db233deb_study,0,0,1,0,c6e92e59a0ae_image,NaN,none 1 0 0 1 1,0fd2db233deb_study


In [12]:
label_data[label_data['id_y']=='04cc2f7f4c4b_image']

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
375,0fd2db233deb_study,0,0,1,0,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb_study


In [13]:
label_data.id_y.value_counts(ascending=False)

1f0419f1f5f8_image    1
cbfd2079791c_image    1
86a2f241cf70_image    1
ae21e4271637_image    1
e547eeebb69f_image    1
                     ..
35da4a5a0201_image    1
ff0743bee789_image    1
df176d43ac18_image    1
cfdedbaaaee1_image    1
bf1f75117093_image    1
Name: id_y, Length: 6334, dtype: int64

In [14]:
label_data.drop_duplicates(subset=['id_x'],inplace=True)

In [15]:
label_data.shape

(6054, 9)

In [16]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
0,00086460a852_study,0,1,0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study
1,000c9c05fd14_study,0,0,0,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study


In [17]:
label_data.columns

Index(['id_x', 'Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance', 'id_y', 'boxes',
       'label', 'StudyInstanceUID'],
      dtype='object')

In [18]:
for i in ['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']:
    label_data[i] = label_data[i].replace(1,i) 

In [19]:
label_data['target'] = label_data['Negative for Pneumonia'].astype('str') + label_data['Atypical Appearance'].astype('str') + \
                        label_data['Typical Appearance'].astype('str') + label_data['Indeterminate Appearance'].astype('str')

In [20]:
label_data['target'] = label_data['target'].apply(lambda x: re.sub('0','',x))

In [21]:
label_data['target'].value_counts()

Typical Appearance          2855
Negative for Pneumonia      1676
Indeterminate Appearance    1049
Atypical Appearance          474
Name: target, dtype: int64

In [22]:
label_data.sample()

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID,target
3720,97692c000b63_study,0,Typical Appearance,0,0,20db49453bf4_image,"[{'x': 574.28399, 'y': 657.98967, 'width': 695...",opacity 1 574.28399 657.98967 1270.23242 1234....,97692c000b63_study,Typical Appearance


In [23]:
label_data['id_y'] = label_data['id_y'].apply(lambda x: re.sub('_image','.jpg',x))

In [24]:
label_data['path'] = 'gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train/' + label_data['id_y'].astype('str')

In [25]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID,target,path
0,00086460a852_study,0,Typical Appearance,0,0,65761e66de9f.jpg,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study,Typical Appearance,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...
1,000c9c05fd14_study,0,0,0,Atypical Appearance,51759b5579bc.jpg,NaN,none 1 0 0 1 1,000c9c05fd14_study,Atypical Appearance,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...


In [26]:
label_data.target.value_counts()

Typical Appearance          2855
Negative for Pneumonia      1676
Indeterminate Appearance    1049
Atypical Appearance          474
Name: target, dtype: int64

In [27]:
label_data['target'] = label_data['target'].apply(lambda x: re.sub(' ','_',x))

### Size prediction

In [43]:
object_tag = []
for i,j in enumerate(label_data.id_y):
    try:
        image = cv2.imread('kaggle_data/train/'+j,0)
        x_max,y_max = image.shape
        list_len = label_data.label.iloc[i].split(' ')
        iter_run = len(list_len)//6
        ls = []
        if list_len[0]=='opacity':
            for k in range(0,iter_run):
                if k == 0:
                    new_ls = []
#                     new_ls.append(list_len[0])
                    new_ls.append(float(list_len[2])/y_max)
                    new_ls.append(float(list_len[3])/x_max)
                    new_ls.append(float(list_len[4])/y_max)
                    new_ls.append(float(list_len[5])/x_max)
                    ls.append(new_ls)
                else:
                    new_ls = []
                    new_ls.append(float(list_len[2+(6*k)])/y_max)
                    new_ls.append(float(list_len[3+(6*k)])/x_max)
                    new_ls.append(float(list_len[4+(6*k)])/y_max)
                    new_ls.append(float(list_len[5+(6*k)])/x_max)
                    ls.append(new_ls)
#                     print(ls)
        else:
#             ls.append(list_len[0])
#             ls.append(0)
#             ls.append(0)
#             ls.append(1)
#             ls.append(1)
            new_ls = []
            new_ls.append(0)
            new_ls.append(0)
            new_ls.append(1)
            new_ls.append(1)
            ls.append(new_ls)
    except:
        pass
    object_tag.append(ls)
#     break

In [45]:
label_data['image_detection'] = object_tag

In [50]:
label_data = label_data.explode('image_detection')

In [51]:
subset_data = label_data[['path','target','image_detection']]

In [52]:
subset_data

,path,target,image_detection
0,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.258947951850521, 0.2731804635193133, 0.3783..."
0,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.7347394502335609, 0.3639082489270386, 0.853..."
1,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Atypical_Appearance,"[0, 0, 1, 1]"
2,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Negative_for_Pneumonia,"[0, 0, 1, 1]"
3,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Negative_for_Pneumonia,"[0, 0, 1, 1]"
...,...,...,...
6331,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.4800375758575198, 0.34176191269841266, 0.79..."
6332,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.14961774771609276, 0.18234397854077253, 0.3..."
6332,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.5364447189037245, 0.36751030042918453, 0.84..."
6333,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,"[0.22347811440677964, 0.42983565229885057, 0.3..."


In [53]:
subset_data = subset_data.reset_index(drop=True)

In [54]:
bounding_box = subset_data['image_detection'].apply(pd.Series)

In [55]:
objection_df = subset_data[['path','target']]

In [58]:
objection_df = pd.concat([objection_df,bounding_box[[0,1,2,3]]],axis=1)

In [59]:
objection_df[['x1','y1','x2','y2']] = objection_df[[0,1,2,3]]

In [60]:
objection_df[['path','target','x1','y1','x2',1,2,3,0,'y2']]

,path,target,x1,y1,x2,1,2,3,0,y2
0,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.258948,0.273180,0.378313,0.273180,0.378313,0.551346,0.258948,0.551346
1,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.734739,0.363908,0.853270,0.363908,0.853270,0.611166,0.734739,0.611166
2,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Atypical_Appearance,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
3,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Negative_for_Pneumonia,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
4,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Negative_for_Pneumonia,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
9548,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.480038,0.341762,0.799436,0.341762,0.799436,0.939111,0.480038,0.939111
9549,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.149618,0.182344,0.335190,0.182344,0.335190,0.677824,0.149618,0.677824
9550,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.536445,0.367510,0.849565,0.367510,0.849565,0.717411,0.536445,0.717411
9551,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...,Typical_Appearance,0.223478,0.429836,0.392291,0.429836,0.392291,0.800253,0.223478,0.800253


In [61]:
objection_df['set'] = 'UNASSIGNED'

In [64]:
new_df = objection_df.loc[objection_df.target!='Negative_for_Pneumonia']

In [65]:
new_df[['set','path','target','x1','y1','x2',1,2,3,0,'y2']].to_csv('./kaggle_data/object_detection.csv',index=False,header=True)

#### transferring data to storage

In [67]:
!gsutil -m cp -r /home/jupyter/kaggle_data/object_detection.csv gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/

Copying file:///home/jupyter/kaggle_data/object_detection.csv [Content-Type=text/csv]...
/ [1/1 files][  1.8 MiB/  1.8 MiB] 100% Done                                    
Operation completed over 1 objects/1.8 MiB.                                      
